# WEEK4：使用 pytorch 搭建简易神经网络

## 4.1 搭建神经网络的一般流程
1. 定义一个拥有可学习参数的神经网络

2. 遍历训练数据集
3. 处理输入数据使其流经神经网络
4. 计算损失值
5. 将网络参数的梯度进行反向传播
6. 以一定的规则更新网络的权重

## 4.2 `torch.nn`介绍
- `torch.nn` 是 pytorch 中一个重要的包，可以用来搭建神经网络

- `torch.nn` 依赖于 `autograd` 包来定义模型，并对它们自动求导

## 4.3 搭建神经网络
### 4.3.1 首先定义一个神经网络

In [40]:
# 导入若干工具包
import torch
import torch.nn as nn
import torch.nn.functional as F
 
 
# 定义一个简单的网络类
class Net(nn.Module):  # 一个初始化函数
    def __init__(self):
        super(Net, self).__init__()
        # 定义第一层卷积神经网络，输入通道维度=1，输出通道维度=6，卷积和大小3*3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 定义第二层卷积神经网络，输入通道维度=6，输出通道维度=16，卷积和大小3*3
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 定义三层全连接网络
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
 
    def forward(self, x):
        # 在（2， 2）的池化窗口下执行最大池化操作
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
 
    # 维度扁平化
    def num_flat_features(self, x):
        # 计算size，除了第0个维度上的batch_size
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
 

### 4.3.2 输出神经网络

In [41]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 4.3.3 通过 `net.parameters()` 来获得模型中所有的可训练参数

In [42]:
params = list(net.parameters())#用list进行封装
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


### 4.3.4 输入图像
- 假设图像的输入尺寸为 $32×32$：

In [43]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0929,  0.1267,  0.0840, -0.0446,  0.0175,  0.0274,  0.0458, -0.0680,
          0.0267, -0.0472]], grad_fn=<AddmmBackward0>)


- 得到输出张量后，接着进行梯度归零和反向传播。

- 注意：`torch.nn`只支持小批量处理 (mini-batches）。整个 `torch.nn` 包只支持小批量样本的输入，不支持单个样本的输入。比如：`nn.Conv2d` 接受一个4维的张量，即 $nSamples × nChannels × Height × Width$。如果是一个单独的样本，只需要使用 `input.unsqueeze(0)`  来添加一个“假的”批大小维度。

In [44]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 4.3.5 损失函数
- 损失函数的输入是： (output,target) ，然后通过损失函数计算出一个数值来评估 output 和 target 之间的差距。

- torch.nn 提供多种不同的损失函数可供使用，例如 nn.MSELoss，它是通过计算均方差损失来评估 output 和 target 之间的差距。

In [45]:
output = net(input)
target = torch.randn(10)
# 改变 target 的形状为二维张量，为了和 output 匹配
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(0.4173, grad_fn=<MseLossBackward0>)


In [46]:
print(loss.grad_fn)# MSELoss
print(loss.grad_fn.next_functions[0][0])# Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])# ReLU

### 4.3.6 反向传播

- 在Pytorch中执行反向传播非常简便，全部的操作就是 `loss.backward()`

- 在执行反向传播之前，要先将梯度清零，否则梯度会在不同的批次数据之间被累加

In [47]:
#梯度清零
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
#反向传播
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0033, -0.0004,  0.0099,  0.0035,  0.0018,  0.0031])


### 4.3.7 更新网络参数

- 更新参数最简单的算法就是SGD（随机梯度下降）

- 具体的算法公式表达式为：$weight = weight - learning rate × gradient$

1. 传统python代码实现随机梯度下降

In [48]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

2. 使用Pytorch官方推荐的标准代码如下：

In [49]:
# 首先导入优化器的包，optim中包含若干常用的优化算法，比如SGD，Adam等
import torch.optim as optim

# 通过opotim创建优化器对象
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 将优化器执行梯度清零的操作
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
# 对损失值执行反向传播的操作
loss.backward()
# 参数的更新通过一行标准代码来执行
optimizer.step()